<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/verbal_nominal_EP_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
import pandas as pd
import re
import unicodedata
from pathlib import Path

# === Параметры — при необходимости поменяйте пути/имена ===
FIRST_PATH  = "polylex_maj.xlsx"   # первая таблица (с выражениями для слияния)
SECOND_PATH = "polylexfle.xlsx"          # вторая таблица (с примерами)
EXPR_COL    = "expression"                   # имя колонки с выражением (в обеих таблицах одинаковое)
EXAMPLE_COL = "Exemple en contexte" # имя колонки с примерами (во второй таблице)
TEST2_COL_1   = "test2_1"             # имя первой колонки для классификации
TEST2_COL_2   = "test2_2"             # имя второй колонки для классификации

# === Функции нормализации (без unidecode) ===
def normalize_expr(s: str) -> str:
    if not isinstance(s, str):
        return ""
    # Нормализация Unicode и пробелов, нижний регистр
    s = unicodedata.normalize("NFKC", s).strip().casefold()
    # Сжать множественные пробелы
    s = re.sub(r"\s+", " ", s)
    return s

# === Чтение таблиц ===
def read_excel_any(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Файл не найден: {path}")
    return pd.read_excel(p)

df_first  = read_excel_any(FIRST_PATH)
df_second = read_excel_any(SECOND_PATH)

# Проверим наличие необходимых колонок
required_cols_first = [EXPR_COL, TEST2_COL_1]
# Add test2_2 to required columns only if it exists in the dataframe
if TEST2_COL_2 in df_first.columns:
    required_cols_first.append(TEST2_COL_2)

for name, df, cols in [("первая", df_first, required_cols_first), ("вторая", df_second, [EXPR_COL, EXAMPLE_COL])]:
    for col in cols:
        if col not in df.columns:
            raise ValueError(f"В {name} таблице не найдена колоonne '{col}'. Найдены: {list(df.columns)}")


# === Классификация выражений в первой таблице ===
def classify_expression_type_from_test2(row):
    """Classifies expression as nominal or verbal based on test2_1 and test2_2 values."""
    test2_1_value = row.get(TEST2_COL_1)
    test2_2_value = row.get(TEST2_COL_2)

    if isinstance(test2_1_value, str):
        if "Test_ID_OUI" in test2_1_value or "Test_ID_NON" in test2_1_value:
            return "nominal"
        elif test2_1_value.strip() == "----------":
            # Check test2_2 if test2_1 is "----------"
            if test2_2_value is None or (isinstance(test2_2_value, str) and not test2_2_value.strip()):
                return "N/A" # Set to N/A if test2_2 is empty
            else:
                # If test2_2 is not empty, use the original verbal logic (or refine if needed)
                 if isinstance(test2_2_value, str):
                    if "Test_ID_OUI" in test2_2_value or "Test_ID_NON" in test2_2_value:
                        return "nominal" # Check test2_2 for nominal indicators
    # Default to verbal if test2_1 doesn't fit nominal criteria and test2_2 doesn't either (or doesn't exist/is empty)
    return "verbal"


# Apply the classification function row-wise
df_first['expression_type'] = df_first.apply(classify_expression_type_from_test2, axis=1)


# === Нормализация ключа и агрегация примеров во второй таблице ===
df_second["_key"]  = df_second[EXPR_COL].apply(normalize_expr)

# Агрегируем примеры из df_second
agg_examples = (
    df_second[df_second[EXAMPLE_COL].notna()] # Исключаем строки, где колонка с примерами пуста
    .groupby("_key")[EXAMPLE_COL]
    .apply(list)
    .reset_index(name="examples_from_second")
)

# === Подготовка первой таблицы и merge ===
# Теперь используем df_first with the new 'expression_type' column
df_first["_key"] = df_first[EXPR_COL].apply(normalize_expr)

# left join: берём ВСЕ выражения из первой таблицы (включая тип), подтягиваем примеры из агрегированной второй таблицы
# Ensure TEST2_COL_2 is included if it exists in df_first for the merge step if needed for classification logic
# But we will drop it before saving.
cols_to_merge = [EXPR_COL, 'expression_type', "_key"]
if TEST2_COL_2 in df_first.columns:
    cols_to_merge.append(TEST2_COL_2)


df_merged = (
    df_first[cols_to_merge] # Include columns needed for merge and potential classification checks
    .merge(agg_examples, on="_key", how="left")
    .drop(columns=["_key"])
)

# === Удаляем колонку test2_2 before saving if it exists ===
if TEST2_COL_2 in df_merged.columns:
    df_merged = df_merged.drop(columns=[TEST2_COL_2])


# превращаем список примеров в строку (если удобнее для Excel)
def fmt_examples(lst):
    if not isinstance(lst, list) or len(lst) == 0:
        return None
    # разделим " | " — можно поменять на "\n" если хотите переносы строк
    return " | ".join(lst)

df_merged["examples_joined"] = df_merged["examples_from_second"].apply(fmt_examples)

# === Удаляем среднюю колонку перед сохранением ===
df_merged = df_merged.drop(columns=["examples_from_second"])


# === Сохраняем результат ===
out_xlsx = "merged_expressions_with_types_and_examples.xlsx" # Updated output filename
out_csv  = "merged_expressions_with_types_and_examples.csv"  # Updated output filename
df_merged.to_excel(out_xlsx, index=False)
df_merged.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"Готово. Сохранено:\n- {out_xlsx}\n- {out_csv}")

# Для быстрого просмотра первых строк
display(df_merged.head(10))

Готово. Сохранено:
- merged_expressions_with_types_and_examples.xlsx
- merged_expressions_with_types_and_examples.csv


,expression,expression_type,examples_joined
0,abonnement mensuel,nominal,"En fait, elle paye un abonnement mensuel qui l..."
1,aboyer (le chien),verbal,Les chiens aboient et les chats miaulent.
2,accent aigu,nominal,Le roman Anna Karénine s'écrit avec un accent ...
3,accent américain,nominal,"D’abord, notre audience est Française à 95%, d..."
4,acquisition des connaissances,nominal,Je m’explique : on peut parfaitement vérifier ...
5,acquérir la nationalité,verbal,On peut également acquérir la nationalité par ...
6,action humanitaire,nominal,Connais-tu des actions humanitaires ?
7,activité humaine,nominal,Les activités humaines détruisent l’environnem...
8,adresser la parole,verbal,Dois-je enfin lui adresser la parole ?
9,agence de publicité,nominal,L’agence de publicité où elle commence est à l...


In [ ]:
import pandas as pd
import math
import os

# Assuming df_merged is already loaded from a previous step and contains 'expression_type'

# Define the number of rows per chunk
chunk_size = 100

# Create directories to save the chunks if they don't exist
nominal_chunks_dir = "expression_chunks/nominal"
verbal_chunks_dir = "expression_chunks/verbal"

if not os.path.exists(nominal_chunks_dir):
    os.makedirs(nominal_chunks_dir)
if not os.path.exists(verbal_chunks_dir):
    os.makedirs(verbal_chunks_dir)

# Split the DataFrame into nominal and verbal
df_nominal = df_merged[df_merged['expression_type'] == 'nominal'].copy()
df_verbal = df_merged[df_merged['expression_type'] == 'verbal'].copy()
df_n_a = df_merged[df_merged['expression_type'] == 'N/A'].copy()


# Function to split a DataFrame into chunks and save
def split_and_save_chunks(df, output_dir, type_name):
    num_chunks = math.ceil(len(df) / chunk_size)
    print(f"Splitting {len(df)} {type_name} expressions into {num_chunks} chunks.")
    for i in range(num_chunks):
        start_row = i * chunk_size
        end_row = (i + 1) * chunk_size
        df_chunk = df.iloc[start_row:end_row]

        output_filename = os.path.join(output_dir, f"{type_name}_part_{i+1}.xlsx")
        df_chunk.to_excel(output_filename, index=False)
        print(f"Saved {output_filename}")
    print(f"Finished splitting {type_name} expressions.")


# Split and save nominal chunks
split_and_save_chunks(df_nominal, nominal_chunks_dir, "nominal")

# Split and save verbal chunks
split_and_save_chunks(df_verbal, verbal_chunks_dir, "verbal")

# Handle N/A entries (optional: save them or print a message)
if not df_n_a.empty:
    print(f"\nFound {len(df_n_a)} expressions classified as N/A. These were not chunked with nominal or verbal.")
    # Optional: Save N/A entries to a separate file
    # df_n_a.to_excel("expression_chunks/n_a_expressions.xlsx", index=False)
    # print("Saved N/A expressions to expression_chunks/n_a_expressions.xlsx")

print("\nFinished splitting all expressions into nominal and verbal chunks.")

Splitting 708 nominal expressions into 8 chunks.
Saved expression_chunks/nominal/nominal_part_1.xlsx
Saved expression_chunks/nominal/nominal_part_2.xlsx
Saved expression_chunks/nominal/nominal_part_3.xlsx
Saved expression_chunks/nominal/nominal_part_4.xlsx
Saved expression_chunks/nominal/nominal_part_5.xlsx
Saved expression_chunks/nominal/nominal_part_6.xlsx
Saved expression_chunks/nominal/nominal_part_7.xlsx
Saved expression_chunks/nominal/nominal_part_8.xlsx
Finished splitting nominal expressions.
Splitting 414 verbal expressions into 5 chunks.
Saved expression_chunks/verbal/verbal_part_1.xlsx
Saved expression_chunks/verbal/verbal_part_2.xlsx
Saved expression_chunks/verbal/verbal_part_3.xlsx
Saved expression_chunks/verbal/verbal_part_4.xlsx
Saved expression_chunks/verbal/verbal_part_5.xlsx
Finished splitting verbal expressions.

Finished splitting all expressions into nominal and verbal chunks.
